# Notebook séance 11

Les [dictionnaires](https://python.sdv.univ-paris-diderot.fr/13_dictionnaires_tuples_sets/) en Python.

Extension de l'exercice sur le parsing d'un fichier Genbank. Voir la section **Nouvelles questions**.

### Extraire les coordonnées des gènes à partir d'un fichier Genbank

Dans une séquence stockée au format GenBank, les gènes sont donnés par les lignes du type :
```txt
     gene            266..21555
```
débutant par le mot-clé `gene` et suivi par un couple de positions, séparé par `..`.

#### Q1.

Ecrire une fonction `genbank2gene` qui étant donné un fichier au format GenBank retourne la liste de couples de positions de l'ensemble des gènes. 
Typiquement :
```cython
>>> genbank2gene("sars-cov-2.gb")
[ (266, 21555), (21563, 25384), (25393, 26220), (26245, 26472), (26523, 27191), (27202, 27387), (27394, 27759), (27756, 27887), (27894, 28259), (28274, 29533), (29558, 29674) ]
```
Il pourrait être utile de se créer une fonction qui étant donné une ligne (une chaîne de caractères) de type `gene` d'un fichier GenBank et retrurne le couple de positions.
```cython
>>> get_positions("     gene            266..21555")
(266, 21555)
```

On pourra tester sur le gène [BRCA2](https://www.ncbi.nlm.nih.gov/nuccore/NM_000059.3) qui ne contient bien sûr qu'un seul gène puis sur le génome du [SARS-CoV-2](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2) 

#### Q2. 

On remarque que sous la *feature* `gene` on trouve une annotation donnant le nom du gène :
```
     gene            266..21555
                     /gene="ORF1ab"
```

Modifiez votre fonction pour qu'elle retourne un dictionnaire qui associe à chaque nom de gène ses positions sur la séquence (on supposera qu'il n'y a pas deux gènes de même nom dans une séquence). 
Typiquement :
```cython
>>> genbank2gene("sars-cov-2.gb")
{ 'ORF1ab': (190,255), 'S': (21563, 25384), ... }
```
Là aussi créer une fonction utilitaire qui transforme une chaîne de caractères de type `/gene` pourrait s'avérer utile.

#### Q3. 

En fait l'annotation `gene` est un peu plus complexe. Un gène peut être positionné sur le brin négatif. Dans ce cas l'annotation a cette forme :
```
     gene            complement(3575721..3577061)
```
et parfois un gène peut être découpé en exons :
```
     gene            join(3583038..3583427,3584205..3584309)
```

Modifiez votre fonction pour qu'elle prenne en compte les gènes sur le brin négatif. Le couple déterminant les positions d'une gène dans le dictionnaire sera alors constitué des deux positions dans l'ordre inverse (la plus grande avant la plus petite, afin de se souvenir qu'on est sur le brin négatif).

Par exemple pour
```
     gene            complement(3575721..3577061)
```
on obtiendra le couple
```
(3577061, 3575721)
```

Testez sur le génome complet d'[E.Coli](https://www.ncbi.nlm.nih.gov/nuccore/NC_000913.3)

#### Q4.

Proposez une manière de stocker les gènes multiexoniques. Modifiez votre fonction pour les prendre en compte.

#### Q5.

Ecrire une fonction qui prend en entrée un fichier Genbank et produit autant de fichiers qu'il y a de gènes, chaque fichier contenant la séquence du gène, i.e. la concaténation des exons. Attention que si le gène est sur le brin négatif, alors il faudra enregistrer dans le fichier la séquence complémentaire inversée.

### Nouvelles questions

Dans les nouvelles questions on aborde une structuration des données différentes.

Au lieu par exemple de convenir que les positions sont données sous  forme de couple on va nommer les positions. Ceci permet une écriture plus lisible du code et va permettre de glisser vers une modélisation orientée objet des données qu'on traite.

#### Q6.

On change maintenant la représentation des gènes d'une séquence. Au lieu de produire un dictionnaire indexé par les noms des gènes, on va produire une liste de gènes, chaque gène étant modélisé par un dictionnaire dont les clés décriront les caractéristiques du gène.

Par exemple le gène ORF1ab cité plus haut sera modélisé par :
```python
{ 'name' : 'ORF1ab', 'begin' : 190, 'end' : 255, strand : '+', exons : [ (190,255) ] }
```
et le gène yaiX de *E.Coli* sera modélisé par :
```python
{ 'name' : 'yaiX', 'begin' : 380844, 'end' : 382872, strand : '-', exons : [ (380844,381260), (382591,382872) ] }
```

AInsi, pour obtenir la longueur d'un gène (dans le cas monoexonique), on aurait écrit dans la précédente représentation :
```python
genes['ORF1ab'][1] - genes['ORF1ab'][0] + 1
```
on écrira maintenant :
```python
# en admettant que genes[0] contienne { 'name' : 'ORF1ab', 'begin' : 190, 'end' : 255, strand : '+', exons : [ (190,255) ] }
orf1ab = genes[0] 
orf1ba['end'] - orf1ab['begin'] + 1
```

Ecrire une nouvelle version de la fonction `genbank2gene` qui produise une liste de tels dictionnaires.


#### Q7.

Allons encore plus loin. Maintenant que nous avons traité les gènes, nous pourrions traiter n'importe quelle annotation du fichier GenBank. L'objectif est désormais de stocker toutes les *features* dans une liste. On modélisera ces *features* comme une liste de dictionnaires. Dans un premier temps, chaque dictionnaire contiendra un type, des coordonnées de debut et de fin et le strand (on traite la partie *Location* de chaque *feature*).

Par exemple, on obtiendra pour Sars-Cov-2 :
```python
[ { 'type' : 'source', 'begin' : 1, 'end' : 29903, 'strand' : '+' },
  { 'type' : "5'UTR", 'begin' : 1, 'end' : 265, 'strand' : '+' },
  { 'type' : 'gene', 'begin' : 266, 'end' : 21555, 'strand' : '+' },
  { 'type' : 'CDS', 'begin' : 266, 'end' : 21555, 'strand' : '+' },
  { 'type' : 'mat_peptide', 'begin' : 266, 'end' : 805, 'strand' : '+' },
  ... ]
```  

Ecrire une fonction `genbank2features` capable de produire une telle liste. Une nouvelle fois, l'écrire de sous-fonctions peut permettre d'aboutir à une écriture plus claire du code.

In [3]:
def genbank2features(filename):
    dico={}
    list_dico=[]
    list_final=[]
    list_transition=[]
    list_remove=[]
    a = 0
    list_type=[]
    list_positions=[]
    #Stockage des informations
    with open(filename,'r') as file:
        line=file.readline()
        #on passe à ligne suivante jusqu'a l'endroit des gene, cds etc
        while not line.startswith("FEATURES"):
            line=file.readline()
        line=file.readline() #aller à la ligne après features
        while not line.startswith("ORIGIN"):
            if line[5] !=' ':
                list_type.append(line[5:21])
                list_positions.append(line[21:37])
            line = file.readline()
#Nous ne nous occupons pas des exons sur plusieurs genes : on les retire
        for i in range(0,len(list_type)):
            if "join" in list_positions[i]:
                list_remove.append(i)
            list_positions[i] = list_positions[i].strip()
            list_type[i] = list_type[i].strip()
        for i in list_remove:
            list_type.remove(list_type[i])
            list_positions.remove(list_positions[i])
#Stockage des type par couple (debut, fin)
        for s in list_positions:
            list_transition = s.split('..')
            list_final.append((list_transition[0], list_transition[1]))
        i=0
#Présentation en liste de dictionnaires
        while i != len(list_type):
            dico = {'type': list_type[i], 'begin': list_final[i][0], 'end': list_final[i][1]}
            list_dico.append(dico)
            i+=1
        return list_dico

In [4]:
genbank2features('sars-cov-2.gb')

[{'type': 'source', 'begin': '1', 'end': '29903'},
 {'type': "5'UTR", 'begin': '1', 'end': '265'},
 {'type': 'gene', 'begin': '266', 'end': '21555'},
 {'type': 'mat_peptide', 'begin': '266', 'end': '805'},
 {'type': 'mat_peptide', 'begin': '806', 'end': '2719'},
 {'type': 'mat_peptide', 'begin': '2720', 'end': '8554'},
 {'type': 'mat_peptide', 'begin': '8555', 'end': '10054'},
 {'type': 'mat_peptide', 'begin': '10055', 'end': '10972'},
 {'type': 'mat_peptide', 'begin': '10973', 'end': '11842'},
 {'type': 'mat_peptide', 'begin': '11843', 'end': '12091'},
 {'type': 'mat_peptide', 'begin': '12092', 'end': '12685'},
 {'type': 'mat_peptide', 'begin': '12686', 'end': '13024'},
 {'type': 'mat_peptide', 'begin': '13025', 'end': '13441'},
 {'type': 'mat_peptide', 'begin': 'join(13442', 'end': '1346'},
 {'type': 'mat_peptide', 'begin': '18040', 'end': '19620'},
 {'type': 'mat_peptide', 'begin': '19621', 'end': '20658'},
 {'type': 'mat_peptide', 'begin': '20659', 'end': '21552'},
 {'type': 'CDS',

#### Q8.

Compléter la fonction précédente pour traiter la partie *Qualifiers* de chaque *feature*. 

Les qualifiers sont généralement de la forme `/nom=valeur`. Les informations complémentaires sont stockées dans un dictionnaire dont chaque clé est le nom du *qualifier* et la valeur ... sa valeur. Attention certains qualifiers comme `/ribosomal_slippage` n'ont pas de valeur associée, on leur associera alors la valeur `True`.

On produira maintenant :

```python
[ { 'type' : 'source', 'begin' : 1, 'end' : 29903, 'strand' : '+', 'informations' : { 'organism' : "Severe acute respiratory syndrome coronavirus 2", 'mol_type' : "genomic RNA", ... } },
  { 'type' : "5'UTR", 'begin' : 1, 'end' : 265, 'strand' : '+', 'informations' : None },
  { 'type' : 'gene', 'begin' : 266, 'end' : 21555, 'strand' : '+', 'informations' : { 'gene' : "ORF1ab", 'locus_tag' : "GU280_gp01", 'db_xref' : "GeneID:43740578" } },
  ... ]
```  



#### Q9.

Ecrire des fonctions utilitaires permettant de manipuler les features. Chacune de ces fonctions prend en paramètre une *feature*, i.e. un dictionnaire  :
- `type`
- `length`
- `strand` (1 ou -1 suivant le strand)
- `sequence` (qui prend en plus en paramètre la chaîne de caractère représentant la séquence)
- `is_gene` (vrai ou faux suivant que c'est un gène)
- `name` (retourne None si ce n'est pas un gène, le nom du gène sinon)

```shell
>>> type({'type' : 'gene', 'begin' : 266, 'end' : 21555, 'strand' : '+', 'informations' : { 'gene' : "ORF1ab", 'locus_tag' : "GU280_gp01", 'db_xref' : "GeneID:43740578" })
'gene'
>>> length({'type' : 'gene', 'begin' : 266, 'end' : 21555, 'strand' : '+', 'informations' : { 'gene' : "ORF1ab", 'locus_tag' : "GU280_gp01", 'db_xref' : "GeneID:43740578" })
21290
>>> strand({'type' : 'gene', 'begin' : 266, 'end' : 21555, 'strand' : '+', 'informations' : { 'gene' : "ORF1ab", 'locus_tag' : "GU280_gp01", 'db_xref' : "GeneID:43740578" })
1
>>> is_gene({'type' : 'gene', 'begin' : 266, 'end' : 21555, 'strand' : '+', 'informations' : { 'gene' : "ORF1ab", 'locus_tag' : "GU280_gp01", 'db_xref' : "GeneID:43740578" })
True
>>> name({'type' : 'gene', 'begin' : 266, 'end' : 21555, 'strand' : '+', 'informations' : { 'gene' : "ORF1ab", 'locus_tag' : "GU280_gp01", 'db_xref' : "GeneID:43740578" })
'ORF1ab'
>>> name({ 'type' : "5'UTR", 'begin' : 1, 'end' : 265, 'strand' : '+', 'informations' : None })
None
>>> name({ 'type' : 'source', 'begin' : 1, 'end' : 29903, 'strand' : '+', 'informations' : { 'organism' : "Severe acute respiratory syndrome coronavirus 2", 'mol_type' : "genomic RNA", ... } })
None
```

#### Q10.

Ecrire une fonction `mean_length` qui prend en paramètre un type et retourne la longueur moyenne de l'ensemble des annotations de ce type.

#### Q11.

Se servir des fonctions précédemment écrites pour établir si il y a une différence de distribution des 3-mers entre les régions couvertes par des gènes et les autres. Je vous laisse faire preuve d'imagination pour savoir comment comparer deux distributions de k-mers.